In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from fit_model import fit_linear_regression_model
from cov_impute import cov_estimate, cov_pilot_estimate
from scipy.stats import norm
from calculate_weights import calculate_optimal_weights
from sklearn.model_selection import train_test_split

In [3]:
data_impute = pd.read_csv('../data/data_full_final.csv') # Read the impute data
df = pd.read_csv("../data/pilot_final.csv") # Read the pilot data

In [5]:
data_pilot = pd.merge(df, data_impute.rename(columns = {"positive": "positive_pred", "strong": "strong_pred"}), how = "inner", on = ["id", "time_index"]) # Construct the pilot data
data_pilot.head(5)

,id,time_index,positive,strong,positive_pred1,strong_pred1,time,positive_pred,strong_pred,danmaku,...,start_hour,morning,noon,afternoon,evening,weekend,fans_log,like_cum_log,zifu_log,duration_log
0,7036548193076726568,745,0,0,0.043978,0.129394,2021-12-01 11:05:48 - 2021-12-01 11:05:53,0.0,0.0,0,...,11,0,1,0,0,0,12.826152,7.081709,0.000000,8.634289
1,7036549296094841615,1200,1,1,0.036339,0.300978,2021-12-01 11:50:32 - 2021-12-01 11:50:37,1.0,1.0,2,...,11,0,1,0,0,0,12.076402,7.628031,2.890372,8.804847
2,7036550372629957410,676,0,0,0.028739,0.055494,2021-12-01 11:07:22 - 2021-12-01 11:07:27,0.0,0.0,0,...,11,0,1,0,0,0,8.739376,5.303305,0.000000,8.569953
3,7036550372629957410,925,0,0,0.032281,0.113497,2021-12-01 11:28:07 - 2021-12-01 11:28:12,0.0,0.0,0,...,11,0,1,0,0,0,8.740177,5.560682,0.000000,8.569953
4,7036554379562453796,973,0,0,0.039068,0.072499,2021-12-01 11:54:48 - 2021-12-01 11:54:53,0.0,0.0,0,...,11,0,1,0,0,0,8.920790,6.492240,0.000000,8.622245


In [6]:
like_pilot = np.log(1+data_pilot['like']) # The response
covariates_list = ['positive','strong',  
                    "fans_log", 'like_cum_log', "zifu_log"
                ] # The list of covariates
covariates_pilot = data_pilot[covariates_list] # Take the subvector

In [7]:
model_pilot_sm = sm.OLS(like_pilot, sm.add_constant(covariates_pilot)) # Linear regression model
res_pilot = model_pilot_sm.fit() # Fit the model
res_pilot.summary() # Present the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   like   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     138.4
Date:                Wed, 21 May 2025   Prob (F-statistic):          6.17e-140
Time:                        12:53:36   Log-Likelihood:                -5345.9
No. Observations:                6568   AIC:                         1.070e+04
Df Residuals:                    6562   BIC:                         1.074e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.3229      0.053     -6.130      0.000      -0.426      -0.220
positive         0.0139      0.029      0.472      0.637      -0.044       0.072
strong           0.0435      0.018      2.377      0.017       0.008       0.079
fans_log         0.0178      0.005      3.679      0.000       0.008       0.027
like_cum_log     0.0809      0.004     19.239      0.000       0.073       0.089
zifu_log         0.0673      0.006     10.862      0.000       0.055       0.079
==============================================================================
Omnibus:                      815.559   Durbin-Watson:                   1.747
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1149.090
Skew:                           1.020   Prob(JB):                    3.01e-250
Kurtosis:                       2.800   Cond. No.                         97.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
W_pilot = np.load("../data/W_pilot_final.npy") # Load the vector of W0

In [10]:
Z0 = data_pilot[['positive', 'strong']].to_numpy() # Get the Z vector for pilot sample
Z0_hat = data_pilot[['positive_pred1', 'strong_pred1']].to_numpy() # Get the Zhat vector for pilot sample
n = Z0.shape[0] # The pilot sample size
W0 = np.hstack([np.ones((n,1)),W_pilot]) # Get the W0 vector for pilot sample
r = W0.shape[1] # The dimension of W0
p = 2 # The dimension of Z

In [13]:
U0 = np.hstack([data_pilot[['positive', 'strong']].to_numpy(), sm.add_constant(data_pilot[covariates_list[2:]]).to_numpy()]) # The U vector of pilot sample
U0_hat = np.hstack([data_pilot[['positive_pred1', 'strong_pred1']].to_numpy(), sm.add_constant(data_pilot[covariates_list[2:]]).to_numpy()]) # The Uhat vector of pilot sample
q = U0.shape[1] - p # The dimension of X
Y0 = np.log(1+data_pilot['like']).to_numpy() # The response of pilot sample
U_hat = np.hstack([data_impute[['positive', 'strong']].to_numpy(), sm.add_constant(data_impute[covariates_list[2:]]).to_numpy()]) # The Uhat vector of impute sample
Y = np.log(1+data_impute['like']).to_numpy() # The response of whole sample
N = Y.shape[0] # The whole sample size

In [14]:
model_pilot = fit_linear_regression_model(U0, Y0) # Fit the pilot model
model_impute = fit_linear_regression_model(U_hat, Y) # Fit the impute model
sigmaimp_hat, sigma_hat_pilot = cov_estimate(W0, U0, U0_hat, U_hat, Y0, Y, model_pilot) # Compute the covariance estimator for impute model

In [15]:
# The order is Z1, Z2, 1, X1, X2, X3
se_impute = np.sqrt(np.diag(sigmaimp_hat)) # Compute the standard error
p_val_impute = ((1 - norm.cdf(abs(model_impute.coef_ / se_impute))) * 2) # Compute the pvalue
model_impute.coef_, se_impute, p_val_impute # Show the regression results

(array([ 0.04039732,  0.0380381 , -0.34761427,  0.02128227,  0.07877732,
         0.06517042]),
 array([0.00823006, 0.0025126 , 0.00262344, 0.0002432 , 0.00020665,
        0.00030443]),
 array([9.17716571e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00]))